### Resources:

* http://www.twilightlandscapes.com/IRAFtutorial/IRAFintro_06.html

In [ ]:
import os

# Delete some directories/files from previous runs.

os.system("rm -rf login.cl database pyraf uparm")
os.system("mkiraf")


import astropy.io.fits as fits
%pylab
%matplotlib inline

# Dark subtract and flat field the spectra
from os.path import expanduser
home = expanduser("~/")
data_dir=home+"/data_home/ATC2017/23june_Spectra/"


In [ ]:
ls ~/data_home/ATC2017/23june_Spectra/

In [ ]:
import glob 
wideslit_flats=glob.glob(data_dir+"/23june_wideslit-*domeflat.fit")
darks=glob.glob(data_dir+"23june_wideslit_11grating-*dark1.fit")
darks[1],wideslit_flats[0]

In [ ]:
# The name of the science file.
target="vega.flattened.fits"
filename = target+ '.fits'
extracted_filename = target+'.ms.fits'
calibrated_filename = target+'.calib.fits'

In [ ]:
dark =  fits.getdata(darks[0])
#dark_exptime = dark[0].header["EXPTIME"]
flat =  fits.getdata(wideslit_flats[0])



## Use astropy to do initial data reduction

In [ ]:
plt.figure(figsize=[8,3])
vega_fname = data_dir+"/23june_wideslit_11grating-003Vega.fit"
vega= fits.open(vega_fname)
plt.subplot(121)
plt.imshow(vega[0].data)
plt.colorbar()

#arcturus[0].header["flatfile"]= wideslit_flats[0]
vega[0].header["darkfile"]= darks[0]

plt.subplot(122)
vega[0].data = vega[0].data-np.float_(dark) 
#float_ is necessary to change the byte order, e.g.
#https://github.com/mperrin/poppy/issues/213#issuecomment-289166554
plt.imshow(np.log10(vega[0].data))#,vmax=6e5,vmin=0 ) 
plt.colorbar()

vega.writeto("vega.flattened.fits",clobber=True)

## Use pyraf to analyze the spectrum

In [ ]:
# Now load IRAF
import pyraf.iraf as iraf

# Load the packages we might need.

iraf.noao(_doprint=0)
iraf.onedspec(_doprint=0)
iraf.twodspec(_doprint=0)
iraf.apextract(_doprint=0)
iraf.unlearn(iraf.apall)
iraf.unlearn(iraf.autoidentify)


In [ ]:
# Delete previous results.

os.system("rm "+extracted_filename+" "+calibrated_filename)

# Make sure that the dispersion axis is in the header.

iraf.hedit(images=[filename], fields=["DISPAXIS"], value=["1"], add="Yes")

Commands for graphics windows:
1. `w` to enter zoom mode 
2. `e` in the bottom left and top right corner of the box you want zoom in to.
3. `w a` reset the zoom
4. `m` centroid and define center of the analysis aperture
5. `b` advance to setting the background
6. `s` select background
7. `f` fit background
8. `q` to advance to the next stage
9. `:or` increase order of polynomial fit


In [ ]:
# Run the spectral extraction program.

iraf.apextract.setParam("dispaxis", "1")

iraf.apall(input=filename, find="No", recenter="No", resize="No",interactive="No")
filename

In [ ]:

iraf.identify(filename[:-5], 
              section="line 265 285")

In [ ]:
iraf.autoidentify(filename[:-5], coordli=home+"/Downloads/HgNe(1).dat",
              section="line 265 285",
                  interactive='yes',
                  crval=5000,
                  cdelt=1.2,
                  fwidth=10)


In [ ]:
# Tell the extracted spectrum what the wavelength solutions are.

iraf.hedit(images=[extracted_filename], fields=["REFSPEC1"], \
        value=[filename], add="Yes")

iraf.dispcor(input=extracted_filename, output=calibrated_filename)

# Plot the extracted spectrum?

In [ ]:
iraf.splot(calibrated_filename)

In [ ]:
#when in doubt:
iraf.flpr()
iraf.flpr()
iraf.flpr()


In [ ]:
fits.getdata("vega.flattened.fits.ms.fits")

In [ ]:
plt.plot(fits.getdata("vega.flattened.fits.ms.fits"))
plt.ylim([0,7e5])


In [ ]:
plt.plot(fits.getdata("vega.flattened.fits.calib.fits"))
plt.ylim([0,7e5])

In [ ]:
A0V=fits.getdata(home+"/Downloads/pickles_uk_9.fits")
plot(A0V["WAVELENGTH"],A0V["FLUX"])

In [ ]:
from scipy.interpolate import interp1d
A0V_interpolator = interp1d(A0V["WAVELENGTH"],A0V["FLUX"])
